## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
# already set up on Expanse; toggle for colab

# pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] pyfaidx

Set path prefix for Expanse

In [1]:
path_prefix = "/expanse/lustre/projects/nji102/sgriesmer/"

Import SNP datasets

In [2]:
import pandas as pd

abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/SNP_PRVCS_PAT_0_1738-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')
abs_diff_plus_log_odds_scores_norm.head(), abs_diff_plus_log_odds_scores_norm.shape


(   BroadDnd41CtcfUniPk151-ran  BroadDnd41Ezh239875UniPk151-ran  \
 0                   -0.224394                        -0.461671   
 1                   -0.224228                        -0.516556   
 2                   -0.224098                        -0.333868   
 3                   -0.224193                        -0.371427   
 4                   -0.224354                        -0.476290   
 
    BroadGm12878CtcfUniPk151-ran  BroadGm12878Ezh239875UniPk151-ran  \
 0                     -0.272129                          -0.822471   
 1                     -0.272071                           0.266989   
 2                     -0.271749                          -0.849166   
 3                     -0.089786                          -0.704137   
 4                     -0.250339                           2.322652   
 
    BroadH1hescChd1a301218aUniPk151-ran  BroadH1hescCtcfUniPk151-ran  \
 0                             0.114470                    -0.240770   
 1                      

Choose test set for run

In [3]:
import xgboost

In [4]:
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot

In [5]:
X=abs_diff_plus_log_odds_scores_norm[0:1738]

In [6]:
X

,BroadDnd41CtcfUniPk151-ran,BroadDnd41Ezh239875UniPk151-ran,BroadGm12878CtcfUniPk151-ran,BroadGm12878Ezh239875UniPk151-ran,BroadH1hescChd1a301218aUniPk151-ran,BroadH1hescCtcfUniPk151-ran,BroadH1hescEzh239875UniPk151-ran,BroadH1hescJarid1aab26049UniPk151-ran,BroadH1hescRbbp5a300109aUniPk151-ran,BroadHelas3CtcfUniPk151-ran,...,UwMcf7CtcfUniPk151-ran.1,UwNb4CtcfUniPk151-ran.1,UwNhdfneoCtcfUniPk151-ran.1,UwNhekCtcfUniPk151-ran.1,UwNhlfCtcfUniPk151-ran.1,UwRptecCtcfUniPk151-ran.1,UwSaecCtcfUniPk151-ran.1,UwSknshraCtcfUniPk151-ran.1,UwWerirb1CtcfUniPk151-ran.1,UwWi38CtcfUniPk151-ran.1
0,-0.224394,-0.461671,-0.272129,-0.822471,0.114470,-0.240770,-0.321725,-0.517211,-0.421976,-0.257994,...,-0.443892,-0.323766,-0.368216,-0.397702,-0.371787,-0.170180,-0.541176,-0.357181,-0.289044,-0.405426
1,-0.224228,-0.516556,-0.272071,0.266989,3.203063,-0.240770,-0.310654,-0.506043,1.170008,-0.257977,...,-0.417354,-0.315919,-0.381523,-0.346374,-0.376833,-0.319919,-0.140645,-0.358484,-0.285417,-0.145505
2,-0.224098,-0.333868,-0.271749,-0.849166,-0.366690,-0.240770,-0.320375,-0.524898,-0.392277,-0.257941,...,-0.379402,-0.326086,-0.192139,-0.251094,-0.249033,-0.324480,0.166988,-0.353359,-0.262350,-0.366049
3,-0.224193,-0.371427,-0.089786,-0.704137,-0.439970,-0.237890,-0.320804,-0.527675,-0.422983,-0.257977,...,-0.425844,-0.155586,-0.278957,-0.226195,0.523562,0.320484,-0.547248,-0.335464,-0.275627,-0.402639
4,-0.224354,-0.476290,-0.250339,2.322652,-0.427354,-0.240768,-0.222454,0.061876,0.484678,-0.153564,...,-0.425258,-0.320962,-0.330111,-0.389199,-0.381161,-0.166626,-0.223527,-0.336442,-0.289492,1.227873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733,-0.222968,-0.232788,-0.242138,-0.467199,0.838026,-0.240746,-0.321721,-0.388706,-0.184898,-0.253702,...,-0.198594,-0.197522,-0.207631,-0.193369,-0.122823,-0.182345,0.016334,-0.348478,-0.232228,-0.186224
1734,-0.224327,-0.543424,-0.272022,0.226540,-0.408600,-0.240768,0.072354,-0.363062,-0.308156,-0.241612,...,-0.412964,-0.324997,-0.094758,-0.358858,-0.372440,-0.383902,0.104727,-0.355155,-0.280701,-0.435271
1735,-0.224314,-0.288421,-0.271317,-0.666786,-0.441242,-0.240769,-0.319253,-0.522392,-0.072093,-0.257741,...,-0.438679,-0.306988,-0.081668,-0.181159,-0.069852,-0.323038,-0.104314,-0.295356,-0.288906,-0.362358
1736,1.976589,-0.486019,1.740339,-0.402627,-0.441121,1.057054,-0.321840,0.192769,-0.409122,-0.112477,...,0.441317,1.121061,0.303402,0.690241,-0.217326,0.325158,-0.162566,-0.085255,-0.235584,0.603584


In [7]:
col_lab = list(range(0,1380))
X.set_axis(col_lab, axis="columns", inplace=True)

In [8]:
X

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.224394,-0.461671,-0.272129,-0.822471,0.114470,-0.240770,-0.321725,-0.517211,-0.421976,-0.257994,...,-0.443892,-0.323766,-0.368216,-0.397702,-0.371787,-0.170180,-0.541176,-0.357181,-0.289044,-0.405426
1,-0.224228,-0.516556,-0.272071,0.266989,3.203063,-0.240770,-0.310654,-0.506043,1.170008,-0.257977,...,-0.417354,-0.315919,-0.381523,-0.346374,-0.376833,-0.319919,-0.140645,-0.358484,-0.285417,-0.145505
2,-0.224098,-0.333868,-0.271749,-0.849166,-0.366690,-0.240770,-0.320375,-0.524898,-0.392277,-0.257941,...,-0.379402,-0.326086,-0.192139,-0.251094,-0.249033,-0.324480,0.166988,-0.353359,-0.262350,-0.366049
3,-0.224193,-0.371427,-0.089786,-0.704137,-0.439970,-0.237890,-0.320804,-0.527675,-0.422983,-0.257977,...,-0.425844,-0.155586,-0.278957,-0.226195,0.523562,0.320484,-0.547248,-0.335464,-0.275627,-0.402639
4,-0.224354,-0.476290,-0.250339,2.322652,-0.427354,-0.240768,-0.222454,0.061876,0.484678,-0.153564,...,-0.425258,-0.320962,-0.330111,-0.389199,-0.381161,-0.166626,-0.223527,-0.336442,-0.289492,1.227873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733,-0.222968,-0.232788,-0.242138,-0.467199,0.838026,-0.240746,-0.321721,-0.388706,-0.184898,-0.253702,...,-0.198594,-0.197522,-0.207631,-0.193369,-0.122823,-0.182345,0.016334,-0.348478,-0.232228,-0.186224
1734,-0.224327,-0.543424,-0.272022,0.226540,-0.408600,-0.240768,0.072354,-0.363062,-0.308156,-0.241612,...,-0.412964,-0.324997,-0.094758,-0.358858,-0.372440,-0.383902,0.104727,-0.355155,-0.280701,-0.435271
1735,-0.224314,-0.288421,-0.271317,-0.666786,-0.441242,-0.240769,-0.319253,-0.522392,-0.072093,-0.257741,...,-0.438679,-0.306988,-0.081668,-0.181159,-0.069852,-0.323038,-0.104314,-0.295356,-0.288906,-0.362358
1736,1.976589,-0.486019,1.740339,-0.402627,-0.441121,1.057054,-0.321840,0.192769,-0.409122,-0.112477,...,0.441317,1.121061,0.303402,0.690241,-0.217326,0.325158,-0.162566,-0.085255,-0.235584,0.603584


In [9]:
import pandas as pd

snp_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/PAT_dataset_XY-named-equal.csv", sep=',')


In [10]:
y = snp_prvcs['label'][0:1738]

In [11]:
y

0       1
1       1
2       1
3       1
4       1
       ..
1733    0
1734    0
1735    0
1736    0
1737    0
Name: label, Length: 1738, dtype: int64

In [12]:
y[860:870]

860    1
861    1
862    1
863    1
864    1
865    1
866    1
867    1
868    1
869    0
Name: label, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
test_size = 0.3
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [14]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score

model1 = XGBClassifier() 
model2 = XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5) 
train_model1 = model1.fit(X_train, y_train) 
train_model2 = model2.fit(X_train, y_train) 
pred1 = train_model1.predict(X_test) 
pred2 = train_model2.predict(X_test) 
print("Accuracy_model1: %.4f" % (accuracy_score(y_test, pred1))) 
print("Accuracy_model2: %.4f" % (accuracy_score(y_test, pred2))) 
print("F1_model1: %.4f" % (f1_score(y_test, pred1)))
print("F1_model2: %.4f" % (f1_score(y_test, pred2)))
print("MCC_model1: %.4f" % (matthews_corrcoef(y_test, pred1)))
print("MCC_model2: %.4f" % (matthews_corrcoef(y_test, pred2)))
print("ROC_AUC_model1: %.4f" % (roc_auc_score(y_test, pred1))) # don't know about call: y_score vs. pred
print("ROC_AUC_model2: %.4f" % (roc_auc_score(y_test, pred2))) # don't know about call: y_score vs. pred


Accuracy_model1: 0.6322
Accuracy_model2: 0.6494
F1_model1: 0.6265
F1_model2: 0.6460
MCC_model1: 0.2717
MCC_model2: 0.3053
ROC_AUC_model1: 0.6355
ROC_AUC_model2: 0.6525


In [15]:
pred2.sum()

240

In [16]:
model3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=7)

train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)
print("Accuracy_model3: %.4f" % (accuracy_score(y_test, pred3)))
print("F1_model3: %.4f" % (f1_score(y_test, pred3)))
print("MCC_model3: %.4f" % (matthews_corrcoef(y_test, pred3)))
print("ROC_AUC_model3: %.4f" % (roc_auc_score(y_test, pred3))) # don't know about call: y_score vs. pred

Accuracy_model3: 0.6705
F1_model3: 0.6641
MCC_model3: 0.3495
ROC_AUC_model3: 0.6742


In [17]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[10],
 'min_child_weight':[1],
 'learning_rate':[0.1],
 'n_estimators':[1000],
 'objective':['binary:logistic']
 #'objective':['binary:logistic','binary:hinge','binary:logitraw']
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(gamma=0, 
                                               subsample=0.8, 
                                               colsample_bytree=0.8, 
                                               nthread=4, 
                                               scale_pos_weight=1, 
                                               seed=7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=5,
                       refit=True)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
print("Accuracy_model4: %.4f" % (accuracy_score(y_test, pred4)))  
print("F1_model4: %.4f" % (f1_score(y_test, pred4)))  
print("MCC_model4: %.4f" % (matthews_corrcoef(y_test, pred4)))  
print("ROC_AUC_model4: %.4f" % (roc_auc_score(y_test, pred4)))  

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Accuracy_model4: 0.6667
F1_model4: 0.6575
MCC_model4: 0.3433
ROC_AUC_model4: 0.6708


In [18]:
len(pred4)

522

In [19]:
gsearch.cv_results_['params'][gsearch.best_index_]

{'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 1000,
 'objective': 'binary:logistic'}

In [20]:
model_deepsea1 = XGBClassifier(
 reg_alpha = 20,
 reg_lambda = 2000,
 eta = 0.1,
 n_estimators=10,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea1 = model_deepsea1.fit(X_train, y_train)
pred_deepsea1 = train_model_deepsea1.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea1)))
print("F1_model_deepsea1: %.4f" % (f1_score(y_test, pred_deepsea1)))
print("MCC_model_deepsea1: %.4f" % (matthews_corrcoef(y_test, pred_deepsea1)))
print("ROC_AUC_model_deepsea1: %.4f" % (roc_auc_score(y_test, pred_deepsea1))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.4693
F1_model_deepsea1: 0.0000
MCC_model_deepsea1: 0.0000
ROC_AUC_model_deepsea1: 0.5000


/cm/shared/apps/spack/0.17.3/cpu/b/opt/spack/linux-rocky8-zen/gcc-8.5.0/anaconda3-2021.05-q4munrgvh7qp4o7r3nzcdkbuph4z7375/lib/python3.8/site-packages/sklearn/metrics/_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [21]:
model_deepsea2 = XGBClassifier(
 reg_alpha = 0,
 reg_lambda = 10,
 eta = 0.1,
 n_estimators=500,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea2 = model_deepsea2.fit(X_train, y_train)
pred_deepsea2 = train_model_deepsea2.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea2)))
print("F1_model_deepsea2: %.4f" % (f1_score(y_test, pred_deepsea2)))
print("MCC_model_deepsea2: %.4f" % (matthews_corrcoef(y_test, pred_deepsea2)))
print("ROC_AUC_model_deepsea2: %.4f" % (roc_auc_score(y_test, pred_deepsea2))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.6782
F1_model_deepsea2: 0.6653
MCC_model_deepsea2: 0.3690
ROC_AUC_model_deepsea2: 0.6831


In [22]:
# Save best model

In [23]:
from joblib import dump, load

best_estimator = train_model_deepsea2
dump(best_estimator, path_prefix + "DNABERT_2/Output_Models/" + "model_xpg_boost_PRVCS_PAT.joblib")

['/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_PAT.joblib']

In [24]:
test_estimator = load('/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_PAT.joblib')

In [25]:
test_pred = test_estimator.predict(X_test)

In [26]:
test_pred.sum()/522

0.43103448275862066

In [27]:
# Load SNP data

In [27]:
SNPlt01_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/lt05_igap_ran_0_10000-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [28]:
# Set model names to numbers

In [29]:
SNPlt01_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [30]:
SNPlt01_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.134340,-0.176669,-0.167205,1.136512,0.598432,-0.162685,-0.172438,0.035790,-0.143567,-0.154109,...,-0.316542,-0.200087,-0.322255,-0.241135,-0.097094,-0.150122,-0.420258,-0.225923,-0.196953,-0.289094
1,-0.134376,-0.366801,-0.167250,-0.652682,-0.361398,-0.162674,-0.203901,-0.443247,-0.298716,-0.154114,...,0.419565,-0.195666,-0.377331,-0.174015,-0.222661,-0.306294,-0.472069,-0.218917,-0.197477,-0.280357
2,-0.023767,-0.197831,-0.167147,0.070671,-0.200981,0.235543,-0.203821,-0.207416,-0.223960,-0.147906,...,2.071424,-0.186225,-0.038674,0.445475,0.074265,0.001689,-0.210508,-0.193521,-0.186807,0.506437
3,-0.134375,-0.437801,-0.167141,-0.155040,-0.285044,-0.162685,-0.203818,-0.269907,-0.297292,-0.154112,...,-0.319017,-0.198726,-0.384140,-0.275544,-0.037049,-0.276440,-0.447298,-0.224284,-0.196686,-0.247948
4,-0.134354,-0.498539,-0.167261,-0.653013,-0.356434,-0.162685,-0.203906,-0.443459,-0.298768,-0.154109,...,-0.278643,-0.200142,-0.378112,-0.200160,-0.197726,-0.271060,-0.457370,-0.226998,-0.196294,-0.271508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.134333,-0.102992,-0.167079,0.438134,-0.358858,-0.162685,-0.203737,5.375807,-0.297380,-0.154109,...,-0.316421,-0.195129,-0.325719,-0.169471,-0.159812,0.021548,-0.315698,-0.225919,-0.197648,-0.276390
9996,-0.130458,-0.029596,-0.127534,1.780980,-0.002129,-0.162602,-0.203783,-0.264749,0.331581,-0.145322,...,0.118872,-0.152362,-0.317824,0.050219,0.247068,-0.077249,-0.193526,-0.225535,-0.169228,0.006952
9997,-0.134323,-0.276844,-0.167105,-0.526303,-0.351880,-0.162685,-0.198805,-0.370694,-0.266654,-0.154111,...,-0.309555,-0.190732,-0.338031,-0.288699,-0.218013,-0.282265,-0.440474,-0.219109,-0.186410,-0.270539
9998,-0.134193,0.185418,-0.165822,-0.172234,-0.355526,-0.162660,-0.203906,-0.435158,-0.297253,-0.153950,...,-0.224890,-0.168852,-0.143209,-0.260668,-0.245686,-0.070232,-0.192810,-0.107638,-0.195652,0.022638


In [31]:
SNPlt01_pred = test_estimator.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [32]:
SNPlt01_pred[0:10000].sum()

9998

In [33]:
SNPgt5_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/gt5_igap_ran_0_10000-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [34]:
SNPgt5_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [35]:
SNPgt5_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.134340,-0.176669,-0.167205,1.136512,0.598432,-0.162685,-0.172438,0.035790,-0.143567,-0.154109,...,-0.316542,-0.200087,-0.322255,-0.241135,-0.097094,-0.150122,-0.420258,-0.225923,-0.196953,-0.289094
1,-0.134376,-0.366801,-0.167250,-0.652682,-0.361398,-0.162674,-0.203901,-0.443247,-0.298716,-0.154114,...,0.419565,-0.195666,-0.377331,-0.174015,-0.222661,-0.306294,-0.472069,-0.218917,-0.197477,-0.280357
2,-0.023767,-0.197831,-0.167147,0.070671,-0.200981,0.235543,-0.203821,-0.207416,-0.223960,-0.147906,...,2.071424,-0.186225,-0.038674,0.445475,0.074265,0.001689,-0.210508,-0.193521,-0.186807,0.506437
3,-0.134375,-0.437801,-0.167141,-0.155040,-0.285044,-0.162685,-0.203818,-0.269907,-0.297292,-0.154112,...,-0.319017,-0.198726,-0.384140,-0.275544,-0.037049,-0.276440,-0.447298,-0.224284,-0.196686,-0.247948
4,-0.134354,-0.498539,-0.167261,-0.653013,-0.356434,-0.162685,-0.203906,-0.443459,-0.298768,-0.154109,...,-0.278643,-0.200142,-0.378112,-0.200160,-0.197726,-0.271060,-0.457370,-0.226998,-0.196294,-0.271508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.134333,-0.102992,-0.167079,0.438134,-0.358858,-0.162685,-0.203737,5.375807,-0.297380,-0.154109,...,-0.316421,-0.195129,-0.325719,-0.169471,-0.159812,0.021548,-0.315698,-0.225919,-0.197648,-0.276390
9996,-0.130458,-0.029596,-0.127534,1.780980,-0.002129,-0.162602,-0.203783,-0.264749,0.331581,-0.145322,...,0.118872,-0.152362,-0.317824,0.050219,0.247068,-0.077249,-0.193526,-0.225535,-0.169228,0.006952
9997,-0.134323,-0.276844,-0.167105,-0.526303,-0.351880,-0.162685,-0.198805,-0.370694,-0.266654,-0.154111,...,-0.309555,-0.190732,-0.338031,-0.288699,-0.218013,-0.282265,-0.440474,-0.219109,-0.186410,-0.270539
9998,-0.134193,0.185418,-0.165822,-0.172234,-0.355526,-0.162660,-0.203906,-0.435158,-0.297253,-0.153950,...,-0.224890,-0.168852,-0.143209,-0.260668,-0.245686,-0.070232,-0.192810,-0.107638,-0.195652,0.022638


In [36]:
SNPgt5_pred = test_estimator.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [37]:
SNPgt5_pred.sum()

9998

In [38]:
# Try other models

In [39]:
pred1_lt01 = train_model1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred2_lt01 = train_model2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred3_lt01 = train_model3.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred4_lt01 = train_model4.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_lt01 = train_model_deepsea1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_lt01 = train_model_deepsea2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [40]:
for i in [
    pred1_lt01,
    pred2_lt01,
    pred3_lt01,
    pred4_lt01,
    pred_deepsea1_lt01,
    pred_deepsea2_lt01,
  ]:
    print(i.sum())

9990
9998
10000
10000
0
9998


In [41]:
pred1_gt5 = train_model1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred2_gt5 = train_model2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred3_gt5 = train_model3.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred4_gt5 = train_model4.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_gt5 = train_model_deepsea1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_gt5 = train_model_deepsea2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [42]:
for i in [
    pred1_gt5,
    pred2_gt5,
    pred3_gt5,
    pred4_gt5,
    pred_deepsea1_gt5,
    pred_deepsea2_gt5,
  ]:
    print(i.sum())

9990
9998
10000
10000
0
9998
